In [ ]:
# ===============================================================
# ===============================================================
# ===       NOTEBOOK FOR SAVING THE HYPERTUNED MODELS         ===
# ===                      FOR SITE SEQ                       ===
# ===============================================================
# ===============================================================

__date__= '30-Oct-22'
__author__='jeremy charlier'
__revised__='17-Nov-22'

"""comments
the encoder class is inherited from
[1]: "CRISPR-Net: A Recurrent Convolutional Network Quantiﬁes
CRISPR Off-Target Activities with Mismatches and Indels", J. Lin et al
https://onlinelibrary.wiley.com/doi/epdf/10.1002/advs.201903562
"""

import sys
path_to_module = 'MODULE_PATH'   # append drive directory to python sys path
sys.path.append(path_to_module)
sys.path.append(path_to_module+'/code/')   # location of python source code
sys.path.append(path_to_module)

In [ ]:
import pickle as pkl
from joblib import dump
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.utils import Bunch
from sklearn.metrics import (
	classification_report, roc_auc_score,
	confusion_matrix, f1_score,
	roc_curve, precision_score, recall_score,
	auc, average_precision_score,
	precision_recall_curve, accuracy_score)
from imblearn.under_sampling import RandomUnderSampler as rus
#
from transferlearning_datapipeline import datapipeline
from simanalysis import getBootStrappedData
from transferlearning_modelpipeline import modelpipeline
import transferlearning_utils as tlrn
import transferlearning_tensorflow_models as tf_models
#
p = print

In [ ]:
def rebalanceData(
    xtrain, ytrain,
    is_sampling = True,
    sampling_ratio = 0.8):
  # change sampling strat for CIRCLE-SEQ (itrain==1)
  # sampling_ratio = 0.2 else 0.8
  if is_sampling:
    xtrainres, ytrainres = rus(
      sampling_strategy = sampling_ratio,
      random_state = 0
    ).fit_resample(xtrain, ytrain)
  else:
    xtrainres, ytrainres = xtrain, ytrain
  return xtrainres, ytrainres
# end function
#
def getFprTpr(estimator, disp = False):
  preds = estimator.yscore
  if len(preds.shape) == 2:
    ypreds = preds[:,1]
  else:
    ypreds = preds
  fprs, tprs, _ = roc_curve(estimator.y_test, ypreds)
  if disp:
    print('false positive rates:\n', fprs)
    print('true positive rates:\n', tprs)
  return fprs, tprs
#

In [ ]:
def urnd(x): return np.round(x, 3);
#
def plotRocCurve(
		estimators,
    icol = 1, disp = False):
  nwpos = 0
  plt.figure(figsize=(8, 6))
  plt.plot([0, 1], [0, 1], 'k--')
  # +++ MLP 1 layer +++
  fprs, tprs = getFprTpr( estimators[nwpos], disp = disp )
  plt.plot(
    fprs, tprs,
    label = 'MLP 1 layer (AUC: %s \u00B1 0.001)' % (urnd(auc(fprs, tprs))))
  # +++ MLP 2 layers +++
  nwpos += 1
  fprs, tprs = getFprTpr( estimators[nwpos], disp = disp )
  plt.plot(
    fprs, tprs,
    label = 'MLP 2 layers (AUC: %s \u00B1 0.001)' % (urnd(auc(fprs, tprs))))
  # +++ RF +++
  nwpos += 1
  fprs, tprs = getFprTpr( estimators[nwpos], disp = disp )
  plt.plot(
    fprs, tprs,
    label = 'RF (AUC: %s \u00B1 0.001)' % (urnd(auc(fprs, tprs))))
  # +++ LR model in position 3 +++
  nwpos += 1
  fprs, tprs = getFprTpr( estimators[nwpos], disp = disp )
  plt.plot(
    fprs, tprs,
    label='LR (AUC: %s \u00B1 0.001)' % (urnd(auc(fprs, tprs))))
  # dllab = [
  #   'FFN3', 'FFN5', 'FFN10',
  #   'CNN3', 'CNN5', 'CNN10',
  #   'LSTM', 'GRU']
  # for nwpos in range(4, 12):
  #   clf = estimators[nwpos]
  #   plt.plot(
  #     clf.fpr, clf.tpr,
  #     label = dllab[nwpos-4] + ' (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ FNN3 model in position 4 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label='FFN3 (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ FNN5 model in position 5 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label='FFN5 (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # FNN10 model in position 6
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label='FFN10 (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ CNN3 model in position 7 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label = 'CNN3 (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ CNN5 model in position 8 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label = 'CNN5 (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ CNN10 model in position 9 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label = 'CNN10 (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ LSTM model in position 10 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label = 'LSTM (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ GRU model in position 11 +++
  nwpos += 1
  clf = estimators[nwpos]
  plt.plot(
    clf.fpr, clf.tpr,
    label = 'GRU (AUC: %s \u00B1 0.001)' % (clf.roc_auc))
  # +++ plot legend +++
  plt.xlabel('False positive rate')
  plt.ylabel('True positive rate')
  plt.legend(loc='best')
  # plt.show()
  plt.savefig('plot1.pdf')
#

In [ ]:
# === VARIABLE INITIALIZATION ===
issavemodels = True # save models (Y=True / N=False)
issklearnmodels = False # for sklearn models
isfnnmodels = False # for FNNs models
iscnnmodels = True # for CNNs models
isrnnmodels = True # for RNNs models
#
# === READ ENCODED DATA ===
is_read_pkl_encoded_data = True
if is_read_pkl_encoded_data:
  f = open(path_to_module+'/data/encoded_data.pkl', 'rb')
  encdata = pkl.load(f)
  f.close()
else:
  encdata = datapipeline()

In [ ]:
# === DATA SITE SEQ ===
is_verbose_training = False
idata = 2
#
data = encdata[idata]
xtrain, xtest, ytrain, ytest = tlrn.dataSplitRF(data)
xtrainres, ytrainres = xtrain, ytrain

In [ ]:
# ========================================================
# ========================================================
#                      SKLEARN MODELS
# ========================================================
# ========================================================
is_mlp = True
is_mlp2 = True
is_rf = True
is_lr = True
if issklearnmodels:
  if is_mlp:
    # === MLP 1 LAYER ===
    print('=== MLP 1 LAYER TRAINING ===')
    clf = MLP(
      hidden_layer_sizes=(94,),
      activation = 'logistic', solver = 'adam',
      learning_rate = 'invscaling',
      early_stopping = False,
      max_iter = 1000, random_state = 0 )
    mlp = modelpipeline(
        clf,
        xtrainres, ytrainres,
        xtest, ytest, is_verbose_training
      ).modelTrain()
    mlp = mlp.modelPredict()
  #
  if is_mlp2:
    # === MLP 2 LAYERS ===
    print('=== MLP 2 LAYERS TRAINING ===')
    clf = MLP(
      hidden_layer_sizes=( 250, 25 ),
      activation = 'logistic', solver = 'adam',
      learning_rate = 'invscaling',
      early_stopping = False,
      max_iter = 1000, random_state = 0
    )
    mlp_2layers = modelpipeline(
        clf,
        xtrainres, ytrainres,
        xtest, ytest, is_verbose_training
      ).modelTrain()
    mlp_2layers = mlp_2layers.modelPredict()
  #
  if is_rf:
    # === RANDOM FOREST ===
    print('=== RANDOM FOREST TRAINING ===')
    clf = RandomForestClassifier(
      n_estimators = 784, criterion = 'entropy',
      n_jobs = -1, random_state = 0)
    rf = modelpipeline(
        clf,
        xtrainres, ytrainres,
        xtest, ytest, is_verbose_training
      ).modelTrain()
    rf = rf.modelPredict()
  #
  if is_lr:
    # === LOGISTIC REGRESSION ===
    print('=== LOGISTIC REGRESSION TRAINING ===')
    clf = LogisticRegression(
      solver = 'sag', penalty = 'l2',
      C = 0.233572, max_iter = 10000,
      n_jobs = -1, random_state = 0)
    lr = modelpipeline(
        clf,
        xtrainres, ytrainres,
        xtest, ytest, is_verbose_training
      ).modelTrain()
    lr = lr.modelPredict()
  #
  if issavemodels:
    print('=== SAVING SKLEARN MODELS ===')
    if is_mlp:
      dump(mlp, path_to_module+'/saved_models/mlp_siteseq.joblib')
    if is_mlp2:
      dump(mlp_2layers, path_to_module+'/saved_models/mlp_2layers_siteseq.joblib')
    if is_rf:
      dump(rf, path_to_module+'/saved_models/rf_siteseq.joblib')
    if is_lr:
      dump(lr, path_to_module+'/saved_models/lr_siteseq.joblib')

In [ ]:
# ========================================================
# ========================================================
#                      FNNS MODELS
# ========================================================
# ========================================================
if isfnnmodels:
  max_iter_training = 25
  # === DATA FOR FFNS ===
  data_test_size = tf_models.define_data_test_size(idata)
  x_train_ffns, x_test_ffns, y_train_ffns, y_test_ffns = tf_models.data_split_for_fnns(
    encdata[idata],
    testsize = data_test_size
  )
  x_boot_ffns, y_boot_ffns = tf_models.bootstrapForHypertuning( encdata[idata] )
  data_dict = {
    'x_train': x_train_ffns, 'y_train': y_train_ffns,
    'x_boot': x_boot_ffns, 'y_boot': y_boot_ffns,
    'x_test': x_test_ffns, 'y_test': y_test_ffns
  }
  # === FFN 3 LAYERS ===
  best_ffn3_roc = 0
  for iter in range(max_iter_training):
    params_grid_ffn3 = {
      'unit_layer_1': 64,
      'unit_layer_2': 100,
      'unit_layer_3': 16,
      'unit_dropout_1': 0,
      'is_batch_normalization_1': True,
      'unit_batch': 32
    }
    ffn3_tmp = tf_models.dlModelsPipeline(
      tf_models.model_ffn3,
      data_dict,
      params_grid_ffn3
    )
    ffn3_tmp.modelTrain()
    ffn3_tmp.modelPredict()
    if ffn3_tmp.roc_auc > best_ffn3_roc:
      best_ffn3_roc = ffn3_tmp.roc_auc
      ffn3 = ffn3_tmp
  print('> best roc auc FFN3:', ffn3.roc_auc)
  #
  # === FFN 5 LAYERS ===
  best_ffn5_roc = 0
  for iter in range(max_iter_training):
    params_grid_ffn5 = {
      'unit_layer_1': 100,
      'unit_layer_2': 8,
      'unit_layer_3': 128,
      'unit_layer_4': 100,
      'unit_layer_5': 32,
      'unit_dropout_1': 0.1,
      'unit_dropout_2': 0.05,
      'is_batch_normalization_1': True,
      'is_batch_normalization_2': True,
      'unit_batch': 32
    }
    ffn5_tmp = tf_models.dlModelsPipeline(
      tf_models.model_ffn5,
      data_dict,
      params_grid_ffn5
    )
    ffn5_tmp.modelTrain()
    ffn5_tmp.modelPredict()
    if ffn5_tmp.roc_auc > best_ffn5_roc:
      best_ffn5_roc = ffn5_tmp.roc_auc
      ffn5 = ffn5_tmp
  print('> best roc auc FFN5:', ffn5.roc_auc)
  #
  # === FFN 10 LAYERS ===
  best_ffn10_roc = 0
  for iter in range(max_iter_training):
    params_grid_ffn10 = {
      'unit_layer_1': 64,
      'unit_layer_2': 128,
      'unit_layer_3': 32,
      'unit_layer_4': 256,
      'unit_layer_5': 32,
      'unit_layer_6': 32,
      'unit_layer_7': 128,
      'unit_layer_8': 128,
      'unit_layer_9': 64,
      'unit_layer_10': 64,
      'unit_dropout_1': 0.15,
      'unit_dropout_2': 0.2,
      'unit_dropout_3': 0.05,
      'unit_dropout_4': 0.15,
      'is_batch_normalization_1': True,
      'is_batch_normalization_2': True,
      'is_batch_normalization_3': True,
      'is_batch_normalization_4': True,
      'is_batch_normalization_5': True,
      'is_batch_normalization_6': True,
      'unit_batch': 64
    }
    ffn10_tmp = tf_models.dlModelsPipeline(
      tf_models.model_ffn10,
      data_dict,
      params_grid_ffn10
    )
    ffn10_tmp.modelTrain()
    ffn10_tmp.modelPredict()
    if ffn10_tmp.roc_auc > best_ffn10_roc:
      best_ffn10_roc = ffn10_tmp.roc_auc
      ffn10 = ffn10_tmp
  print('> best roc auc FFN10:', ffn10.roc_auc)
  #
  # === SAVING FFNS ===
  if issavemodels:
    print('=== SAVING FNNS MODELS ===')
    ffn3.trained_model.save(path_to_module+'/saved_models/ffn3_siteseq')
    ffn5.trained_model.save(path_to_module+'/saved_models/ffn5_siteseq')
    ffn10.trained_model.save(path_to_module+'/saved_models/ffn10_siteseq')

In [ ]:
# ========================================================
# ========================================================
#                      CNNS MODELS
# ========================================================
# ========================================================
if iscnnmodels:
  max_iter_training = 25
  # === DATA FOR CNNS ===
  data_test_size = tf_models.define_data_test_size(idata)
  x_train_cnns, x_test_cnns, y_train_cnns, y_test_cnns = tf_models.data_split_for_cnns(
    encdata[idata],
    testsize = data_test_size
  )
  x_train_cnns = x_train_cnns.reshape(-1, 24, 7, 1)
  x_test_cnns = x_test_cnns.reshape(-1, 24, 7, 1)
  x_boot_cnns, y_boot_cnns = tf_models.bootstrapForHypertuning( encdata[idata] )
  x_boot_cnns = x_boot_cnns.reshape(-1, 24, 7, 1)
  data_dict = {
    'x_train': x_train_cnns, 'y_train': y_train_cnns,
    'x_boot': x_boot_cnns, 'y_boot': y_boot_cnns,
    'x_test': x_test_cnns, 'y_test': y_test_cnns
  }
  # === CNN 3 LAYERS ===
  best_cnn3_roc = 0
  for iter in range(max_iter_training):
    params_grid_cnn3 = {
      'unit_layer_1' : 256,
      'unit_layer_2' : 128,
      'activation_layer_1' : 'softmax',
      'activation_layer_2' : 'tanh',
      'activation_layer_3' : 'tanh',
      'unit_dropout_1' : 0.1,
      'unit_dropout_2' : 0.05,
      'is_batch_normalization_1' : True,
      'is_batch_normalization_2' : True,
      'unit_batch' : 128}
    cnn3_tmp = tf_models.dlModelsPipeline(
      tf_models.model_cnn3,
      data_dict,
      params_grid_cnn3
    )
    cnn3_tmp.modelTrain()
    cnn3_tmp.modelPredict()
    if cnn3_tmp.roc_auc > best_cnn3_roc:
      best_cnn3_roc = cnn3_tmp.roc_auc
      cnn3 = cnn3_tmp
  print('> best roc auc CNN3:', cnn3.roc_auc)
  #
  # === CNN 5 LAYERS ===
  best_cnn5_roc = 0
  for iter in range(max_iter_training):
    params_grid_cnn5 = {
      'unit_layer_1' : 128,
      'unit_layer_2' : 256,
      'unit_layer_3' : 75,
      'unit_layer_4' : 64,
      'activation_layer_1' : 'tanh',
      'activation_layer_2' : 'softmax',
      'activation_layer_3' : 'softmax',
      'activation_layer_4' : 'tanh',
      'activation_layer_5' : 'relu',
      'unit_dropout_1' : 0.15,
      'unit_dropout_2' : 0.1,
      'is_batch_normalization_1' : True,
      'is_batch_normalization_2' : True,
      'is_batch_normalization_3' : True,
      'unit_batch' : 128}
    cnn5_tmp = tf_models.dlModelsPipeline(
      tf_models.model_cnn5,
      data_dict,
      params_grid_cnn5
    )
    cnn5_tmp.modelTrain()
    cnn5_tmp.modelPredict()
    if cnn5_tmp.roc_auc > best_cnn5_roc:
      best_cnn5_roc = cnn5_tmp.roc_auc
      cnn5 = cnn5_tmp
  print('> best roc auc CNN5:', cnn5.roc_auc)
  #
  # === CNN 10 LAYERS ===
  best_cnn10_roc = 0
  for iter in range(max_iter_training):
    params_grid_cnn10 = {
      'unit_layer_1' : 100,
      'unit_layer_2' : 200,
      'unit_layer_3' : 256,
      'unit_layer_4' : 100,
      'unit_layer_5' : 75,
      'unit_layer_6' : 8,
      'unit_layer_7' : 75,
      'unit_layer_8' : 32,
      'unit_layer_9' : 8,
      'activation_layer_1' : 'tanh',
      'activation_layer_2' : 'softmax',
      'activation_layer_3' : 'relu',
      'activation_layer_4' : 'softmax',
      'activation_layer_5' : 'softmax',
      'activation_layer_6' : 'tanh',
      'activation_layer_7' : 'softmax',
      'activation_layer_8' : 'relu',
      'activation_layer_9' : 'softmax',
      'activation_layer_10' : 'relu',
      'unit_dropout_1' : 0.05,
      'unit_dropout_2' : 0.1,
      'unit_dropout_3' : 0.1,
      'unit_dropout_4' : 0,
      'unit_dropout_5' : 0.1,
      'unit_dropout_6' : 0,
      'is_batch_normalization_1' : True,
      'is_batch_normalization_2' : True,
      'is_batch_normalization_3' : True,
      'is_batch_normalization_4' : True,
      'is_batch_normalization_5' : True,
      'is_batch_normalization_6' : True,
      'is_batch_normalization_7' : True,
      'unit_batch' : 256}
    cnn10_tmp = tf_models.dlModelsPipeline(
      tf_models.model_cnn10,
      data_dict,
      params_grid_cnn10
    )
    cnn10_tmp.modelTrain()
    cnn10_tmp.modelPredict()
    if cnn10_tmp.roc_auc > best_cnn10_roc:
      best_cnn10_roc = cnn10_tmp.roc_auc
      cnn10 = cnn10_tmp
  print('> best roc auc CNN10:', cnn10.roc_auc)
  #
  # === SAVING CNNS ===
  if issavemodels:
    print('=== SAVING CNNS MODELS ===')
    cnn3.trained_model.save(path_to_module+'/saved_models/cnn3_siteseq')
    cnn5.trained_model.save(path_to_module+'/saved_models/cnn5_siteseq')
    cnn10.trained_model.save(path_to_module+'/saved_models/cnn10_siteseq')

> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3 training
171/171 [==============================] - 0s 2ms/step
> CNN3

In [ ]:
# ========================================================
# ========================================================
#                      RNNS MODELS
# ========================================================
# ========================================================
if isrnnmodels:
  max_iter_training = 25
  # === DATA FOR RNNS ===
  data_test_size = tf_models.define_data_test_size(idata)
  x_train_rnns, x_test_rnns, y_train_rnns, y_test_rnns = tf_models.data_split_for_cnns(
    encdata[idata], testsize = data_test_size
  )
  x_boot_rnns, y_boot_rnns = tf_models.bootstrapForHypertuning(
    encdata[idata], is_reshape = False
  )
  data_dict = {
    'x_train': x_train_rnns, 'y_train': y_train_rnns,
    'x_boot': x_boot_rnns, 'y_boot': y_boot_rnns,
    'x_test': x_test_rnns, 'y_test': y_test_rnns
  }
  # === LSTM 3 LAYERS ===
  best_lstm3_roc = 0
  for iter in range(max_iter_training):
    params_grid_lstm3 = {
      'unit_layer_1': 200,
      'unit_layer_2': 100,
      'activation_layer_1': 'tanh',
      'activation_layer_2': 'relu',
      'activation_layer_3': 'relu',
      'unit_dropout_1': 0.05,
      'unit_dropout_2': 0.05,
      'is_batch_normalization_1': True,
      'is_batch_normalization_2': True,
      'unit_batch': 32
    }
    lstm3_tmp = tf_models.dlModelsPipeline(
      tf_models.model_lstm3,
      data_dict,
      params_grid_lstm3
    )
    lstm3_tmp.modelTrain()
    lstm3_tmp.modelPredict()
    if lstm3_tmp.roc_auc > best_lstm3_roc:
      best_lstm3_roc = lstm3_tmp.roc_auc
      lstm3 = lstm3_tmp
  print('> best roc auc LSTM3:', lstm3.roc_auc)
  # === GRU 3 LAYERS ===
  best_gru3_roc = 0
  for iter in range(max_iter_training):
    params_grid_gru3 = {
      'unit_layer_1': 256,
      'unit_layer_2': 128,
      'activation_layer_1': 'tanh',
      'activation_layer_2': 'relu',
      'activation_layer_3': 'tanh',
      'unit_dropout_1': 0.05,
      'unit_dropout_2': 0.15,
      'is_batch_normalization_1': True,
      'is_batch_normalization_2': True,
      'unit_batch': 32
    }
    gru3_tmp = tf_models.dlModelsPipeline(
      tf_models.model_gru3,
      data_dict,
      params_grid_gru3
    )
    gru3_tmp.modelTrain()
    gru3_tmp.modelPredict()
    if gru3_tmp.roc_auc > best_gru3_roc:
      best_gru3_roc = gru3_tmp.roc_auc
      gru3 = gru3_tmp
  print('> best roc auc GRU3:', gru3.roc_auc)
  #

  # === SAVING RNNS ===
  if issavemodels:
    print('=== SAVING RNNS MODELS ===')
    lstm3.trained_model.save(path_to_module+'/saved_models/lstm3_siteseq')
    gru3.trained_model.save(path_to_module+'/saved_models/gru3_siteseq')

> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 4ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3ms/step
> LSTM3 training
171/171 [==============================] - 1s 3